In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os,sys
import PIL
import tensorflow as tf
import cv2 
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from PIL import Image
from skimage.io import imread, imshow
from skimage.color import rgb2hsv
from matplotlib import pyplot as plt
from PIL import Image # importing image module 
import numpy as np 
import imageio
import matplotlib.pyplot as plt
from PIL import Image
from PIL import Image




In [3]:
full_dir = "E:/User Document/rotate"
batch_size = 32
img_height = 540
img_width = 180

train_ds = tf.keras.utils.image_dataset_from_directory(
  full_dir,
  validation_split=0.1,#Optional float between 0 and 1, fraction of data to reserve for validation.
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


val_ds = tf.keras.utils.image_dataset_from_directory(
  full_dir,
  validation_split=0.1,#Optional float between 0 and 1, fraction of data to reserve for validation
  subset="validation",
  seed=123,#Optional random seed for shuffling and transformations.
  image_size=(img_height, img_width),
  batch_size=batch_size) #Size of the batches of data. Default: 32. If None, the data will not be batched (the dataset will yield individual samples).
class_names = train_ds.class_names
print(class_names)

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

Found 2527 files belonging to 2 classes.
Using 2275 files for training.
Found 2527 files belonging to 2 classes.
Using 252 files for validation.
['correct', 'rotate']


In [4]:
num_classes = len(class_names)
model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

print(model)

In [5]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 540, 180, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 540, 180, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 270, 90, 16)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 270, 90, 32)       4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 135, 45, 32)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 135, 45, 64)       1

In [6]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/10
72/72 [==============================] - 249s 3s/step - loss: 0.1978 - accuracy: 0.9758 - val_loss: 0.0099 - val_accuracy: 0.9960
Epoch 2/10
72/72 [==============================] - 281s 4s/step - loss: 0.0083 - accuracy: 0.9952 - val_loss: 0.0055 - val_accuracy: 1.0000
Epoch 3/10
72/72 [==============================] - 247s 3s/step - loss: 0.0147 - accuracy: 0.9960 - val_loss: 0.0324 - val_accuracy: 0.9921
Epoch 4/10
72/72 [==============================] - 347s 5s/step - loss: 0.0048 - accuracy: 0.9982 - val_loss: 0.0045 - val_accuracy: 0.9960
Epoch 5/10
72/72 [==============================] - 230s 3s/step - loss: 0.0030 - accuracy: 0.9991 - val_loss: 0.0066 - val_accuracy: 0.9960
Epoch 6/10
72/72 [==============================] - 173s 2s/step - loss: 2.5939e-04 - accuracy: 1.0000 - val_loss: 0.0189 - val_accuracy: 0.9960
Epoch 7/10
72/72 [==============================] - 260s 4s/step - loss: 3.4930e-05 - accuracy: 1.0000 - val_loss: 0.0218 - val_accuracy: 0.9960
Epoch

In [7]:
from PIL import Image
image = Image.open('E:/User Document/rotate/rotate/1.jpg')
image = image.resize((180, 180))
image = np.array(image)
img_tensor = np.expand_dims(image, axis=0)    
model.predict(img_tensor)

ValueError: in user code:

    File "C:\Users\Frammix\anaconda3\lib\site-packages\keras\engine\training.py", line 1621, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\Frammix\anaconda3\lib\site-packages\keras\engine\training.py", line 1611, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Frammix\anaconda3\lib\site-packages\keras\engine\training.py", line 1604, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\Frammix\anaconda3\lib\site-packages\keras\engine\training.py", line 1572, in predict_step
        return self(x, training=False)
    File "C:\Users\Frammix\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Frammix\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 263, in assert_input_compatibility
        raise ValueError(f'Input {

In [ ]:
model.save('E:/User Document/my_model5.h5') 